In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

Задание

Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

1. построить свёрточные архитектуры 

In [4]:
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
from tensorflow.keras import utils as np_utils

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_excel('отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['prepared_Content'] = df['Content'].apply(preprocess_text)
df.head()

,Rating,Content,Date,prepared_Content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро


In [7]:
train_corpus = " ".join(df['prepared_Content'])
train_corpus = train_corpus.lower()

In [8]:
tokens = word_tokenize(train_corpus)

In [9]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [10]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
#tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [11]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]


#x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df['prepared_Content'], df['Rating'], test_size=0.25, random_state=54)

In [13]:
from collections import Counter

words_counter = Counter((word for text in X_train for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

Words count 965


In [14]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

X_train = convert(X_train, word2idx, 1000)
X_test = convert(X_test, word2idx, 1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprec

In [15]:
y_train = y_train.apply (lambda x: x-1)
y_test = y_test.apply (lambda x: x-1)

In [16]:
num_classes = 5
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [17]:
model_cnn = Sequential()
model_cnn.add(Embedding(input_dim=len(word2idx), output_dim=128, input_shape=(X_train.shape[1],)))
model_cnn.add(Conv1D(128, 3))
model_cnn.add(Activation("relu"))
model_cnn.add(GlobalMaxPool1D())
model_cnn.add(Dense(10))
model_cnn.add(Activation("relu"))
model_cnn.add(Dense(num_classes))
model_cnn.add(Activation('softmax'))

In [18]:
model_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 10
batch_size = 1024
print_batch_n = 100

history = model_cnn.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 97s 7s/step - loss: 1.4301 - accuracy: 0.6664 - val_loss: 1.1903 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 96s 7s/step - loss: 1.0634 - accuracy: 0.7084 - val_loss: 0.9301 - val_accuracy: 0.7194
Epoch 3/10
14/14 [==============================] - 94s 7s/step - loss: 0.9138 - accuracy: 0.7084 - val_loss: 0.8641 - val_accuracy: 0.7200
Epoch 4/10
14/14 [==============================] - 100s 7s/step - loss: 0.8192 - accuracy: 0.7287 - val_loss: 0.7606 - val_accuracy: 0.7490
Epoch 5/10
14/14 [==============================] - 94s 7s/step - loss: 0.7230 - accuracy: 0.7622 - val_loss: 0.7017 - val_accuracy: 0.7626
Epoch 6/10
14/14 [==============================] - 95s 7s/step - loss: 0.6720 - accuracy: 0.7747 - val_loss: 0.6751 - val_accuracy: 0.7645
Epoch 7/10
14/14 [==============================] - 94s 7s/step - loss: 0.6471 - accuracy: 0.7804 - val_loss: 0.6682 - val_accuracy: 0.7690
Epoch 8/10
14/14 [=

2. построить различные архитектуры с RNN

In [20]:
model_rnn = Sequential()

model_rnn.add(Embedding(input_dim=len(word2idx), output_dim=128, input_shape=(X_train.shape[1],),
              trainable=True,
              mask_zero=True))
model_rnn.add(Masking(mask_value=0.0))

model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(64, activation='relu'))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(num_classes, activation='sigmoid'))

model_rnn.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_rnn.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 107s 7s/step - loss: 1.1266 - accuracy: 0.6711 - val_loss: 0.9054 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 85s 6s/step - loss: 0.9223 - accuracy: 0.7133 - val_loss: 0.8169 - val_accuracy: 0.7206
Epoch 3/10
14/14 [==============================] - 79s 6s/step - loss: 0.8267 - accuracy: 0.7285 - val_loss: 0.7604 - val_accuracy: 0.7297
Epoch 4/10
14/14 [==============================] - 81s 6s/step - loss: 0.7634 - accuracy: 0.7488 - val_loss: 0.7229 - val_accuracy: 0.7413
Epoch 5/10
14/14 [==============================] - 82s 6s/step - loss: 0.7146 - accuracy: 0.7642 - val_loss: 0.6926 - val_accuracy: 0.7561
Epoch 6/10
14/14 [==============================] - 81s 6s/step - loss: 0.6705 - accuracy: 0.7761 - val_loss: 0.6716 - val_accuracy: 0.7594
Epoch 7/10
14/14 [==============================] - 80s 6s/step - loss: 0.6379 - accuracy: 0.7863 - val_loss: 0.6668 - val_accuracy: 0.7645
Epoch 8/10
14/14 [=

In [27]:
model_lstm = Sequential()

model_lstm.add(Embedding(input_dim=len(word2idx), output_dim=128, input_shape=(X_train.shape[1],),
              trainable=True,
              mask_zero=True))
model_lstm.add(Masking(mask_value=0.0))
model_lstm.add(LSTM(64, recurrent_dropout=0.2))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(num_classes, activation='sigmoid'))

model_lstm.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

In [28]:
history = model_lstm.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 290s 19s/step - loss: 1.2735 - accuracy: 0.6904 - val_loss: 0.9740 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 252s 18s/step - loss: 0.9606 - accuracy: 0.7081 - val_loss: 0.8430 - val_accuracy: 0.7194
Epoch 3/10
14/14 [==============================] - 257s 18s/step - loss: 0.8619 - accuracy: 0.7106 - val_loss: 0.7747 - val_accuracy: 0.7310
Epoch 4/10
14/14 [==============================] - 251s 18s/step - loss: 0.7931 - accuracy: 0.7369 - val_loss: 0.7258 - val_accuracy: 0.7413
Epoch 5/10
14/14 [==============================] - 256s 18s/step - loss: 0.7406 - accuracy: 0.7538 - val_loss: 0.6944 - val_accuracy: 0.7490
Epoch 6/10
14/14 [==============================] - 252s 18s/step - loss: 0.7030 - accuracy: 0.7612 - val_loss: 0.6704 - val_accuracy: 0.7561
Epoch 7/10
14/14 [==============================] - 254s 18s/step - loss: 0.6744 - accuracy: 0.7694 - val_loss: 0.6594 - val_accuracy: 0.7613
Epoch 

In [29]:
model_gru = Sequential()

model_gru.add(Embedding(input_dim=len(word2idx), output_dim=128, input_shape=(X_train.shape[1],),
              trainable=True,
              mask_zero=True))
model_gru.add(Masking(mask_value=0.0))
model_gru.add(GRU(64, recurrent_dropout=0.2))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dropout(0.5))
model_gru.add(Dense(num_classes, activation='sigmoid'))

model_gru.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
history = model_gru.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 203s 14s/step - loss: 1.4140 - accuracy: 0.6466 - val_loss: 1.0009 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 201s 14s/step - loss: 0.9364 - accuracy: 0.7084 - val_loss: 0.8155 - val_accuracy: 0.7194
Epoch 3/10
14/14 [==============================] - 211s 15s/step - loss: 0.8230 - accuracy: 0.7148 - val_loss: 0.7509 - val_accuracy: 0.7290
Epoch 4/10
14/14 [==============================] - 200s 14s/step - loss: 0.7599 - accuracy: 0.7392 - val_loss: 0.7085 - val_accuracy: 0.7452
Epoch 5/10
14/14 [==============================] - 201s 14s/step - loss: 0.7212 - accuracy: 0.7588 - val_loss: 0.6875 - val_accuracy: 0.7574
Epoch 6/10
14/14 [==============================] - 206s 15s/step - loss: 0.6900 - accuracy: 0.7668 - val_loss: 0.6727 - val_accuracy: 0.7587
Epoch 7/10
14/14 [==============================] - 199s 14s/step - loss: 0.6666 - accuracy: 0.7730 - val_loss: 0.6675 - val_accuracy: 0.7606
Epoch 

3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [31]:
model_cnn_rnn = Sequential()
model_cnn_rnn.add(Embedding(input_dim=len(word2idx), output_dim=128, input_shape=(X_train.shape[1],),
              trainable=True,
              mask_zero=True))
model_cnn_rnn.add(Masking(mask_value=0.0))
model_cnn_rnn.add(Conv1D(128, 3))
model_cnn_rnn.add(Activation("relu"))
model_cnn_rnn.add(MaxPooling1D())
model_cnn_rnn.add(Dense(10))
model_cnn_rnn.add(Activation("relu"))
model_cnn_rnn.add(SimpleRNN(64))
model_cnn_rnn.add(Dense(64, activation='relu'))
model_cnn_rnn.add(Dropout(0.5))
model_cnn_rnn.add(Dense(num_classes))
model_cnn_rnn.add(Activation('softmax'))

model_cnn_rnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
history = model_cnn_rnn.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    #callbacks=[tensorboard, early_stopping],
                    validation_data=(X_test, y_test))

Epoch 1/10
14/14 [==============================] - 123s 8s/step - loss: 1.4438 - accuracy: 0.4291 - val_loss: 1.1554 - val_accuracy: 0.7194
Epoch 2/10
14/14 [==============================] - 116s 8s/step - loss: 1.0952 - accuracy: 0.7050 - val_loss: 0.9332 - val_accuracy: 0.7194
Epoch 3/10
14/14 [==============================] - 123s 9s/step - loss: 0.9766 - accuracy: 0.7076 - val_loss: 0.8508 - val_accuracy: 0.7187
Epoch 4/10
14/14 [==============================] - 118s 8s/step - loss: 0.8816 - accuracy: 0.7139 - val_loss: 0.7757 - val_accuracy: 0.7245
Epoch 5/10
14/14 [==============================] - 121s 9s/step - loss: 0.7844 - accuracy: 0.7342 - val_loss: 0.6972 - val_accuracy: 0.7523
Epoch 6/10
14/14 [==============================] - 117s 8s/step - loss: 0.7157 - accuracy: 0.7600 - val_loss: 0.6691 - val_accuracy: 0.7619
Epoch 7/10
14/14 [==============================] - 118s 8s/step - loss: 0.6828 - accuracy: 0.7697 - val_loss: 0.6629 - val_accuracy: 0.7626
Epoch 8/10
14

Наилучший результат у сверточной сети, из RNN сетей лучший результат показала сеть с SimpleRNN, обединёная сеть показала результат хуже чистых сетей.